<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=dc903202dbd2a518890fbb3ca4596798c610ef91c7303525f303faca8207281d
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.1
    Uninstalling protobuf-6.32.1:
      Successfully uninstalled protobuf-6.32.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-10-10 12:15:30
-------------------
qualified stocks: 87
with latest results: 27
still star stocks: 16
-------------------
Initial Investment:  1.23 C
CY Investment:  1.50 C
Reserve:  57.44 K
Current:  1.42 C
-------------------
Today PnL: 51.12 K (0.36%)
Current PnL: -19.91 L (-13.26%)
CY Booked + Current PnL: -8.47 L (-5.64%)
-------------------
Total profit:  1.79 L
Total loss:  -21.70 L
-------------------
Total Booked + Current PnL: 18.51 L (15.03%)
Total Booked PnL: 38.42 L (31.2%)
Curr Year Booked PnL: 11.44 L (8.07%)
Prev Year Booked PnL: 26.98 L (21.91%)
Est FTT:  2.29 C
Est FTT PnL: 87.42 L (61.7%)
Deployed:  1.23 C
Current:  1.42 C
CAGR/XIRR %: 8.45%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118858.0,18354.0,10757.0,-2.30,18.26,9.05,28.96,79.0,1.71,0.84,58.43,MH,ATH,METALS
84,VOLTAS,1530.00,2.42,64.0,X-MC,1.13,214890.0,23148.0,14613.0,3.66,12.07,6.80,19.69,99.0,1.58,1.52,19.69,XY25,NTT,AC
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,212857.0,8357.0,14943.0,2.68,4.09,7.02,11.39,10.0,0.56,1.51,13.40,X40N,ATH,PHARMA
77,TTKPRESTIG,770.00,93.17,39.0,M-SC,1.01,83211.0,-17566.0,17657.0,-0.29,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
14,BLUESTARCO,2080.00,9.63,48.0,X-MC,3.05,189230.0,24560.0,18772.0,-0.01,14.91,9.92,26.31,89.0,1.31,1.34,24.09,X40N,NTT,AC


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,VOLTAS,1530.00,2.42,64.0,X-MC,1.13,214890.0,23148.0,14613.0,3.66,12.07,6.80,19.69,99.0,1.58,1.52,19.69,XY25,NTT,AC
46,KPIGREEN,731.05,-7.87,36.0,H-SC,6.27,109645.0,-15652.0,74581.0,3.15,-12.49,68.02,47.03,141.0,-0.21,0.78,37.29,MH,ATH,POWER
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53535.0,-11471.0,162398.0,2.71,-17.65,303.35,232.18,198.0,-0.07,0.38,2.71,XY24,NTT,MISC
19,CIPLA,1662.74,-20.12,57.0,X-LC,3.67,212857.0,8357.0,14943.0,2.68,4.09,7.02,11.39,10.0,0.56,1.51,13.40,X40N,ATH,PHARMA


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214760.0,9684.0,102892.0,-3.62,4.72,47.91,54.89,52.0,0.09,1.52,29.96,X5K,ATH,METALS
72,TATAELXSI,9161.00,-23.71,47.0,H-MC,6.63,103218.0,-28617.0,70839.0,-2.52,-21.71,68.63,32.03,98.0,-0.40,0.73,15.61,OX40N,NTT,IT
60,SAIL,228.00,45.15,52.0,M-MC,12.70,229181.0,4219.0,163429.0,-2.47,1.88,71.31,74.53,192.0,0.03,1.62,34.73,XY24,BTT,STEEL
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118858.0,18354.0,10757.0,-2.30,18.26,9.05,28.96,79.0,1.71,0.84,58.43,MH,ATH,METALS
82,VALIANTORG,838.00,-290.89,30.0,H-SC,1.68,106148.0,-57457.0,178774.0,-2.18,-35.12,168.42,74.15,139.0,-0.32,0.75,36.06,XR,NTT,CHEMICALS


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UJJIVANSFB,60.0,107.70,63.0,H-SC,13.77,131247.0,-11232.0,30751.0,0.83,-7.88,23.43,13.70,163.0,-0.37,0.93,54.91,OX40N,NTT,BANKS
77,TTKPRESTIG,770.0,93.17,39.0,M-SC,1.01,83211.0,-17566.0,17657.0,-0.29,-17.43,21.22,0.09,245.0,-0.99,0.59,9.32,OX40N,NTT,DURABLES
45,KANSAINER,340.0,-67.97,48.0,H-SC,1.92,220950.0,-48717.0,85044.0,0.06,-18.07,38.49,13.47,138.0,-0.57,1.57,12.35,XY24,NTT,PAINTS
72,TATAELXSI,9161.0,-23.71,47.0,H-MC,6.63,103218.0,-28617.0,70839.0,-2.52,-21.71,68.63,32.03,98.0,-0.40,0.73,15.61,OX40N,NTT,IT
18,CERA,9475.0,-24.58,33.0,H-SC,1.81,137356.0,-38547.0,80573.0,-0.85,-21.91,58.66,23.89,149.0,-0.48,0.97,18.63,OX40N,NTT,CERAMICS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
60,SAIL,228.00,45.15,52.0,M-MC,12.70,229181.0,4219.0,163429.0,-2.47,1.88,71.31,74.53,192.0,0.03,1.62,34.73,XY24,BTT,STEEL
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214760.0,9684.0,102892.0,-3.62,4.72,47.91,54.89,52.0,0.09,1.52,29.96,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154826.0,3881.0,105576.0,1.35,2.57,68.19,72.51,53.0,0.04,1.10,8.75,XR,NTT,IT
79,UNIONBANK,163.00,-13.18,57.0,M-LC,9.18,158327.0,17487.0,27011.0,1.03,12.42,17.06,31.59,66.0,0.65,1.12,42.19,XY24,NTT,BANKS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.95,195418.0,15610.0,96810.0,1.01,8.68,49.54,62.52,88.0,0.16,1.38,45.73,XR,NTT,BANKS


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118858.0,18354.0,10757.0,-2.30,18.26,9.05,28.96,79.0,1.71,0.84,58.43,MH,ATH,METALS
11,BANKINDIA,190.00,-24.22,69.0,H-MC,11.95,195418.0,15610.0,96810.0,1.01,8.68,49.54,62.52,88.0,0.16,1.38,45.73,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214760.0,9684.0,102892.0,-3.62,4.72,47.91,54.89,52.0,0.09,1.52,29.96,X5K,ATH,METALS
86,WIPRO,420.00,-11.98,57.0,M-LC,5.69,154826.0,3881.0,105576.0,1.35,2.57,68.19,72.51,53.0,0.04,1.10,8.75,XR,NTT,IT
37,INDIAMART,4810.62,-54.64,42.0,H-SC,8.73,126691.0,3355.0,128275.0,-0.90,2.72,101.25,106.72,119.0,0.03,0.90,26.73,AR,ATH,MISC


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3465.66,-18.16,35.0,X-LC,5.55,206562.0,-45206.0,98406.0,0.47,-17.96,47.64,21.13,27.0,-0.46,1.46,11.11,X40,ATH,PAINTS
23,DABUR,735.00,-12.30,35.0,X-MC,3.75,237359.0,-10709.0,119107.0,0.82,-4.32,50.18,43.70,102.0,-0.09,1.68,9.51,XY24,BTT,FMCG
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53535.0,-11471.0,162398.0,2.71,-17.65,303.35,232.18,198.0,-0.07,0.38,2.71,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,38.0,X-MC,7.28,222030.0,-41335.0,154688.0,0.53,-15.69,69.67,43.04,84.0,-0.27,1.57,3.08,XY25,ATH,FMCG
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81893.0,-63267.0,139194.0,-0.52,-43.58,169.97,52.31,136.0,-0.45,0.58,9.62,X40N,NTT,FOOTWEAR


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
29,HAVELLS,2069.16,-4.34,47.0,X-MC,0.49,236964.0,-10767.0,85828.0,2.23,-4.35,36.22,30.30,92.0,-0.13,1.68,8.37,X40,ATH,ELECTRICAL
84,VOLTAS,1530.00,2.42,64.0,X-MC,1.13,214890.0,23148.0,14613.0,3.66,12.07,6.80,19.69,99.0,1.58,1.52,19.69,XY25,NTT,AC
73,TATAMOTORS,1065.00,-22.16,43.0,X-LC,1.58,246241.0,-30282.0,140357.0,-0.40,-10.95,57.00,39.81,54.0,-0.22,1.75,18.01,XY24,NTT,AUTO
65,SIEMENS,4671.50,-1.33,48.0,H-LC,1.68,158550.0,-27545.0,75026.0,-0.23,-14.80,47.32,25.51,15.0,-0.37,1.12,16.84,AR,ATH,ELECTRICAL
34,ICICIGI,2260.25,-18.72,49.0,X-MC,1.87,139645.0,3512.0,27608.0,0.23,2.58,19.77,22.86,91.0,0.13,0.99,15.52,X40,ATH,INSURANCE


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53535.0,-11471.0,162398.0,2.71,-17.65,303.35,232.18,198.0,-0.07,0.38,2.71,XY24,NTT,MISC
22,COLPAL,3767.14,-4.98,38.0,X-MC,7.28,222030.0,-41335.0,154688.0,0.53,-15.69,69.67,43.04,84.0,-0.27,1.57,3.08,XY25,ATH,FMCG
83,VBL,671.64,-20.34,43.0,X-LC,10.31,286685.0,-29157.0,141823.0,0.77,-9.23,49.47,35.67,5.0,-0.21,2.03,3.34,X40N,ATH,FMCG
42,ITC,452.00,-38.77,47.0,X-LC,2.40,197116.0,-3022.0,23910.0,0.80,-1.51,12.13,10.44,4.0,-0.13,1.40,3.97,X40,NTT,FMCG
9,BAJAJHFL,181.50,-14.27,39.0,X-MC,4.95,180646.0,-19860.0,117022.0,0.05,-9.91,64.78,48.45,90.0,-0.17,1.28,4.17,X40N,ATH,FINANCE


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
53,QUESS,986.00,-46.01,36.0,X-SC,37.76,53535.0,-11471.0,162398.0,2.71,-17.65,303.35,232.18,198.0,-0.07,0.38,2.71,XY24,NTT,MISC
56,RELAXO,1176.00,-39.88,38.0,X-SC,2.54,81893.0,-63267.0,139194.0,-0.52,-43.58,169.97,52.31,136.0,-0.45,0.58,9.62,X40N,NTT,FOOTWEAR
12,BATAINDIA,2096.00,-34.87,43.0,X-SC,4.42,97359.0,-31311.0,76612.0,0.06,-24.33,78.69,35.20,219.0,-0.41,0.69,11.75,X40,NTT,FOOTWEAR
17,CAMS,5211.76,-4.72,51.0,X-SC,1.44,108282.0,6278.0,37650.0,0.94,6.15,34.77,43.06,122.0,0.17,0.77,24.32,X40N,ATH,MISC
33,HONAUT,58357.33,-22.63,47.0,X-SC,2.14,109095.0,-18843.0,65981.0,0.34,-14.73,60.48,36.84,143.0,-0.29,0.77,11.69,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4406.34,-28.69,50.0,X-LC,12.17,283965.0,-61991.0,130226.0,-1.33,-17.92,45.86,19.73,1.0,-0.48,2.01,4.59,X40,ATH,IT
40,INFY,2275.00,-17.44,57.0,X-LC,7.78,322226.0,9338.0,162337.0,0.23,2.98,50.38,54.87,3.0,0.06,2.28,9.83,X40,BTT,IT
42,ITC,452.00,-38.77,47.0,X-LC,2.40,197116.0,-3022.0,23910.0,0.80,-1.51,12.13,10.44,4.0,-0.13,1.40,3.97,X40,NTT,FMCG
83,VBL,671.64,-20.34,43.0,X-LC,10.31,286685.0,-29157.0,141823.0,0.77,-9.23,49.47,35.67,5.0,-0.21,2.03,3.34,X40N,ATH,FMCG
1,ABB,7934.00,-40.35,48.0,H-LC,7.49,249384.0,-12235.0,131450.0,0.95,-4.68,52.71,45.57,7.0,-0.09,1.77,6.12,AR,NTT,ELECTRICAL


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
21,COFFEEDAY,80.00,-43.66,56.0,L-SC,29.30,84480.0,-29069.0,69122.0,3.04,-25.60,81.82,35.27,268.0,-0.42,0.60,106.77,XR,NTT,HOTELS
49,MASFIN,398.61,-20.05,43.0,H-SC,10.38,90855.0,-7125.0,28728.0,0.55,-7.27,31.62,22.05,152.0,-0.25,0.64,32.06,XR,ATH,FINANCE
50,NATIONALUM,244.55,-40.35,64.0,H-MC,14.27,118858.0,18354.0,10757.0,-2.30,18.26,9.05,28.96,79.0,1.71,0.84,58.43,MH,ATH,METALS
78,UJJIVANSFB,60.00,107.70,63.0,H-SC,13.77,131247.0,-11232.0,30751.0,0.83,-7.88,23.43,13.70,163.0,-0.37,0.93,54.91,OX40N,NTT,BANKS
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,144954.0,-4266.0,148447.0,1.24,-2.86,102.41,96.62,208.0,-0.03,1.03,56.87,XY25,NTT,FINANCE


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
61,SAMMAANCAP,326.00,-184.09,65.0,M-SC,14.38,144954.0,-4266.0,148447.0,1.24,-2.86,102.41,96.62,208.0,-0.03,1.03,56.87,XY25,NTT,FINANCE
69,SURYODAY,240.00,68.86,67.0,H-SC,9.08,154642.0,-24429.0,90156.0,2.03,-13.64,58.30,36.71,135.0,-0.27,1.10,53.44,XR,NTT,BANKS
32,HINDZINC,730.22,31.71,62.0,M-LC,7.74,214760.0,9684.0,102892.0,-3.62,4.72,47.91,54.89,52.0,0.09,1.52,29.96,X5K,ATH,METALS
26,GREENPANEL,537.00,231.40,58.0,M-SC,4.66,148084.0,-34994.0,113965.0,-0.52,-19.11,76.96,43.14,230.0,-0.31,1.05,39.29,XY24,NTT,MISC
68,STARHEALTH,761.00,16.85,68.0,H-SC,7.42,260522.0,-20601.0,153474.0,0.06,-7.33,58.91,47.26,144.0,-0.13,1.85,39.58,XY24,NTT,INSURANCE


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.16
1,25,43.46
2,50,75.19


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.97
LC    32.32
MC    23.73
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.91
X40      14.64
X40N     11.26
XR       10.99
XY25     10.43
AR        8.17
OX40N     7.62
X200      1.75
MH        1.62
X5K       1.52
SR        1.11
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    25.47
X-LC    22.93
X-MC    17.20
M-SC    12.51
H-LC     4.64
H-MC     4.60
X-SC     4.53
M-LC     3.74
M-MC     1.62
L-SC     1.46
L-LC     1.01
L-MC     0.31
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.76,-5.69,40.53
IT,12.85,-15.94,78.73
FINANCE,10.50,-13.85,68.84
MISC,6.91,-17.89,83.22
BANKS,6.41,-10.28,68.30
PAINTS,5.48,-19.50,37.31
ELECTRICAL,5.34,-9.20,47.52
INSURANCE,3.81,-3.11,41.65
AC,3.55,3.38,12.80


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3214436.0,22
XR,1360574.0,14
AR,1177208.0,9
X40,774180.0,10
X40N,646417.0,9
OX40N,570865.0,10
XY25,466029.0,7
SR,265196.0,2
X5K,102892.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3407180.0,25
M-SC,1278670.0,15
X-MC,1107722.0,12
X-LC,1006713.0,13
X-SC,541382.0,6
H-MC,359059.0,4
H-LC,285145.0,3
L-SC,255874.0,3
M-LC,235479.0,3


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1196138.0      6
           AR         807184.0      5
           XR         799622.0      7
M-SC       XY24       687726.0      6
X-MC       XY24       590075.0      4
X-LC       X40        485016.0      5
X-SC       X40N       302372.0      4
H-SC       SR         265196.0      2
           OX40N      264459.0      4
X-MC       X40        212552.0      4
X-LC       X40N       208251.0      3
           XY24       207006.0      2
H-LC       AR         206476.0      2
H-MC       XY24       180653.0      1
X-MC       XY25       169301.0      2
L-SC       XR         167722.0      2
M-SC       AR         163548.0      2
M-MC       XY24       163429.0      1
X-SC       XY24       162398.0      1
M-SC       XY25       148447.0      1
           OX40N      147415.0      4
X-MC       X40N       135794.0      2
M-SC       XR         131534.0      2
X-LC       XY25       106440.0      3
M-LC       XR         105576.0      1
           X5K        102892.0      1
H-MC       XR          96810.0      1
L-SC       OX40N       88152.0      1
H-LC       X200        78669.0      1
X-SC       X40         76612.0      1
H-SC       MH          74581.0      1
H-MC       OX40N       70839.0      1
L-MC       XR          59310.0      1
L-LC       XY25        41841.0      1
M-LC       XY24        27011.0      1
H-MC       MH          10757.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 24.0 seconds
